# Kinesis Firehose
![Kinesis Firehose](img/kinesis_firehose_s3_docs.png)

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
kn_data = boto3.Session().client(service_name='kinesis', region_name=region)
kn_firehose = boto3.Session().client(service_name='firehose', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)

# Download Dataset

In [ ]:
!aws s3 cp 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz' ./data/

In [ ]:
import csv
import pandas as pd

df = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 
                 delimiter='\t', 
                 quoting=csv.QUOTE_NONE,
                 compression='gzip')
df.shape

In [ ]:
df.head(5)

In [1]:
%store -r partition_key

In [2]:
print(partition_key)

CAFEPERSON


In [ ]:
df_star_rating_and_review_body = df[['star_rating', 'review_body']][:100]
df_star_rating_and_review_body.shape

In [ ]:
df_star_rating_and_review_body.head()

In [ ]:
reviews_tsv = df_star_rating_and_review_body.to_csv(sep='\t',
                                                    header=None,
                                                    index=False)

In [ ]:
reviews_tsv

In [ ]:
data_stream = boto3.Session().client(service_name='kinesis', region_name=region)

response = data_stream.put_records(
    Records=[
        {
            'Data': reviews_tsv.encode('utf-8'),
            'PartitionKey': partition_key
        },
    ],
    StreamName=stream_name
)

# TODO:  Firehose to S3